# Conditional AE Hyperparameter Experiments (src-only)

Notebook for running sweeps directly via `src/*` modules (no dependency on `scripts/`).

Per-run artifacts are logged to `experiments/cae/<run_name>/`:
- `config.json`
- `metrics.json`
- `history.csv`
- `checkpoints/cae_best.pt`
- TensorBoard logs in `tb/`


In [7]:
from __future__ import annotations

import json
import os
import random
import sys
from dataclasses import asdict
from datetime import datetime
from itertools import product
from pathlib import Path
from typing import Any

import numpy as np
import pandas as pd
import torch
from IPython.display import display


def find_project_root(start: Path) -> Path:
    start = start.resolve()
    for candidate in [start, *start.parents]:
        if (candidate / 'src').is_dir():
            return candidate
    raise FileNotFoundError('Could not find project root containing src/.')


PROJECT_ROOT = find_project_root(Path.cwd())
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.eval import EvalConfig, evaluate_model
from src.model import ConditionalAE, ModelConfig, count_trainable_parameters
from src.train import SummaryWriter, TrainConfig, build_dataloaders, train_model
from src.utils import (
    get_device,
    load_metadata,
    resolve_urbansound_data_dir,
    save_log_mel_spectrogram,
    unpack_audio,
)

print('PROJECT_ROOT:', PROJECT_ROOT)


PROJECT_ROOT: /Users/kate/untitled folder 10/s26_genai_project


In [8]:
# Paths
DATA_DIR = resolve_urbansound_data_dir(PROJECT_ROOT / 'UrbanSound8K')
AUDIO_DIR = DATA_DIR / 'audio'
AUDIO_UNPACKED_DIR = DATA_DIR / 'audio_unpacked'
SPEC_ROOT = DATA_DIR / 'spectrograms'
EXPERIMENTS_DIR = PROJECT_ROOT / 'experiments' / 'cae'
EXPERIMENTS_DIR.mkdir(parents=True, exist_ok=True)

device = get_device()
metadata = load_metadata(DATA_DIR)

print('DATA_DIR:', DATA_DIR)
print('AUDIO_UNPACKED_DIR:', AUDIO_UNPACKED_DIR)
print('SPEC_ROOT:', SPEC_ROOT)
print('EXPERIMENTS_DIR:', EXPERIMENTS_DIR)
print('Device:', device)
print('Metadata rows:', len(metadata))


DATA_DIR: /Users/kate/untitled folder 10/UrbanSound8K
AUDIO_UNPACKED_DIR: /Users/kate/untitled folder 10/UrbanSound8K/audio_unpacked
SPEC_ROOT: /Users/kate/untitled folder 10/UrbanSound8K/spectrograms
EXPERIMENTS_DIR: /Users/kate/untitled folder 10/s26_genai_project/experiments/cae
Device: cpu
Metadata rows: 8732


In [9]:
# Sweep config (edit for your experiments)
LATENT_DIMS = [64, 128, 256]
EMBED_DIMS = [32]
BASE_CHS = [32]
LRS = [3e-4, 1e-4]
BATCH_SIZES = [64]
SPEC_TS = [176]
VAL_RATIOS = [0.1]

# Data feature hyperparams
N_MELS_VALUES = [128]
N_FFT_VALUES = [2048]
HOP_LENGTH_VALUES = [512]
FMIN = 0.0
FMAX = None

# Training/eval control
EPOCHS = 2
PATIENCE = 12
NUM_WORKERS = 2
SEED = 42
NOISE_STD = 0.5
GRIFFIN_ITERS = 60
MAX_RUNS = 6  # None for full grid
RUN_PREFIX = 'cae_nb'

PREPARE_AUDIO = False
MOVE_AUDIO_FILES = False
PREPARE_SPECS = False
FORCE_RECOMPUTE_SPECS = False
SKIP_EVAL = False

RUN_EXPERIMENTS = True  # switch to True to start training


In [10]:
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def to_jsonable(obj: Any) -> Any:
    if isinstance(obj, Path):
        return str(obj)
    if isinstance(obj, dict):
        return {k: to_jsonable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_jsonable(v) for v in obj]
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    return obj


def write_json(path: Path, payload: dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(to_jsonable(payload), indent=2, ensure_ascii=False))


def save_history(path: Path, train_losses: list[float], val_losses: list[float]) -> None:
    rows = []
    for i, (tr, va) in enumerate(zip(train_losses, val_losses), start=1):
        rows.append({'epoch': i, 'train_l1': tr, 'val_l1': va})
    pd.DataFrame(rows).to_csv(path, index=False)


def make_run_name(run_idx: int, hp: dict[str, Any]) -> str:
    stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return (
        f"{RUN_PREFIX}_{run_idx:03d}_{stamp}"
        f"_lat{hp['latent_dim']}_lr{hp['lr']}_bs{hp['batch_size']}"
        f"_st{hp['spec_t']}_nm{hp['n_mels']}_fft{hp['n_fft']}_hop{hp['hop_length']}"
    )


In [11]:
grid = list(product(
    LATENT_DIMS, EMBED_DIMS, BASE_CHS, LRS, BATCH_SIZES, SPEC_TS, VAL_RATIOS,
    N_MELS_VALUES, N_FFT_VALUES, HOP_LENGTH_VALUES,
))
if MAX_RUNS is not None:
    grid = grid[:MAX_RUNS]

print('Planned runs:', len(grid))
for i, row in enumerate(grid[:10], start=1):
    print(i, row)
if len(grid) > 10:
    print('...')

if not RUN_EXPERIMENTS:
    print('RUN_EXPERIMENTS=False, this is a dry preview.')


Planned runs: 6
1 (64, 32, 32, 0.0003, 64, 176, 0.1, 128, 2048, 512)
2 (64, 32, 32, 0.0001, 64, 176, 0.1, 128, 2048, 512)
3 (128, 32, 32, 0.0003, 64, 176, 0.1, 128, 2048, 512)
4 (128, 32, 32, 0.0001, 64, 176, 0.1, 128, 2048, 512)
5 (256, 32, 32, 0.0003, 64, 176, 0.1, 128, 2048, 512)
6 (256, 32, 32, 0.0001, 64, 176, 0.1, 128, 2048, 512)


In [12]:
run_summaries: list[dict[str, Any]] = []

if RUN_EXPERIMENTS:
    if PREPARE_AUDIO:
        unpack_audio(AUDIO_DIR, AUDIO_UNPACKED_DIR, move_files=MOVE_AUDIO_FILES)

    prepared_specs: set[tuple[int, int, int]] = set()
    multiple_feature_variants = (len(N_MELS_VALUES) * len(N_FFT_VALUES) * len(HOP_LENGTH_VALUES)) > 1

    for run_idx, values in enumerate(grid, start=1):
        (latent_dim, embed_dim, base_ch, lr, batch_size, spec_t, val_ratio, n_mels, n_fft, hop_length) = values

        hp = {
            'latent_dim': latent_dim,
            'embed_dim': embed_dim,
            'base_ch': base_ch,
            'lr': lr,
            'batch_size': batch_size,
            'spec_t': spec_t,
            'val_ratio': val_ratio,
            'n_mels': n_mels,
            'n_fft': n_fft,
            'hop_length': hop_length,
        }

        run_name = make_run_name(run_idx, hp)
        run_dir = EXPERIMENTS_DIR / run_name
        tb_dir = run_dir / 'tb'
        ckpt_path = run_dir / 'checkpoints' / 'cae_best.pt'
        run_dir.mkdir(parents=True, exist_ok=True)

        spec_subdir = SPEC_ROOT / f"nm{n_mels}_fft{n_fft}_hop{hop_length}"
        if multiple_feature_variants or PREPARE_SPECS:
            spec_dir = spec_subdir
        else:
            has_flat_specs = SPEC_ROOT.exists() and any(SPEC_ROOT.glob('*.npy'))
            spec_dir = SPEC_ROOT if has_flat_specs else spec_subdir

        spec_key = (n_mels, n_fft, hop_length)
        if PREPARE_SPECS:
            needs_build = FORCE_RECOMPUTE_SPECS or (spec_key not in prepared_specs) or (not spec_dir.exists())
            if needs_build:
                print(f"[{run_idx}/{len(grid)}] Building spectrograms -> {spec_dir}")
                save_log_mel_spectrogram(
                    metadata=metadata,
                    audio_dir=AUDIO_UNPACKED_DIR,
                    output_dir=spec_dir,
                    n_mels=n_mels,
                    n_fft=n_fft,
                    hop_length=hop_length,
                    fmin=FMIN,
                    fmax=FMAX,
                )
                prepared_specs.add(spec_key)

        print(f"[{run_idx}/{len(grid)}] Run: {run_name}")
        set_seed(SEED + run_idx)

        train_cfg = TrainConfig(
            epochs=EPOCHS,
            lr=lr,
            batch_size=batch_size,
            num_workers=NUM_WORKERS,
            patience=PATIENCE,
            val_ratio=val_ratio,
            seed=SEED + run_idx,
            pin_memory=device.type == 'cuda',
            ckpt_path=ckpt_path,
            log_dir=tb_dir,
        )
        model_cfg = ModelConfig(
            n_classes=int(metadata['classID'].nunique()),
            embed_dim=embed_dim,
            latent_dim=latent_dim,
            base_ch=base_ch,
            spec_h=n_mels,
            spec_t=spec_t,
        )

        run_config = {
            'run_name': run_name,
            'run_dir': run_dir,
            'device': str(device),
            'data_dir': DATA_DIR,
            'audio_unpacked_dir': AUDIO_UNPACKED_DIR,
            'spec_dir': spec_dir,
            'train': asdict(train_cfg),
            'model': asdict(model_cfg),
            'eval': asdict(EvalConfig(batch_size=batch_size, noise_std=NOISE_STD, griffin_lim_iters=GRIFFIN_ITERS)),
        }
        write_json(run_dir / 'config.json', run_config)

        _, train_loader, val_loader = build_dataloaders(
            metadata=metadata,
            spec_dir=spec_dir,
            config=train_cfg,
            spec_t=spec_t,
        )

        model = ConditionalAE(config=model_cfg).to(device)
        params_count = count_trainable_parameters(model)
        print(f"  params={params_count:,} train_batches={len(train_loader)} val_batches={len(val_loader)}")

        train_result = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            config=train_cfg,
        )

        eval_metrics: dict[str, float] = {}
        if not SKIP_EVAL:
            eval_cfg = EvalConfig(batch_size=batch_size, noise_std=NOISE_STD, griffin_lim_iters=GRIFFIN_ITERS)
            eval_metrics = evaluate_model(
                model=model,
                val_loader=val_loader,
                device=device,
                config=eval_cfg,
                checkpoint_path=train_result['ckpt_path'],
            )

        metrics_payload = {
            'params_count': params_count,
            'best_val': train_result['best_val'],
            'best_epoch': train_result['best_epoch'],
            'train_epochs_ran': len(train_result['train_losses']),
            'eval': eval_metrics,
        }
        write_json(run_dir / 'metrics.json', metrics_payload)
        save_history(run_dir / 'history.csv', train_result['train_losses'], train_result['val_losses'])

        tb_writer = SummaryWriter(log_dir=str(tb_dir))
        for key, value in eval_metrics.items():
            tb_writer.add_scalar(f"Eval/{key}", value, int(train_result['best_epoch']))
        hparam_metrics = {'hparam/best_val': float(train_result['best_val'])}
        for key, value in eval_metrics.items():
            hparam_metrics[f"hparam/{key}"] = float(value)
        tb_writer.add_hparams(hp, hparam_metrics)
        tb_writer.close()

        row = {
            'run_name': run_name,
            'run_dir': str(run_dir),
            'spec_dir': str(spec_dir),
            'checkpoint': str(ckpt_path),
            'tb_dir': str(tb_dir),
            'params_count': params_count,
            'best_val': train_result['best_val'],
            'best_epoch': train_result['best_epoch'],
            **hp,
        }
        row.update(eval_metrics)
        run_summaries.append(row)

        print(f"  best_val={train_result['best_val']:.4f}")
        if eval_metrics:
            print(
                f"  eval logmel_l1={eval_metrics['logmel_l1']:.4f} "
                f"mr_stft={eval_metrics['mr_stft']:.4f} "
                f"si_sdr={eval_metrics['si_sdr']:.2f} fad={eval_metrics['fad']:.4f}"
            )
        print(f"  artifacts -> {run_dir}")

    summary_df = pd.DataFrame(run_summaries)
    summary_csv = EXPERIMENTS_DIR / 'summary.csv'
    summary_json = EXPERIMENTS_DIR / 'summary.json'
    summary_df.to_csv(summary_csv, index=False)
    write_json(summary_json, {'runs': run_summaries})

    print('Completed runs:', len(run_summaries))
    print('Summary CSV:', summary_csv)
    print('Summary JSON:', summary_json)
else:
    print('RUN_EXPERIMENTS=False, skipping training loop.')


[1/6] Run: cae_nb_001_20260228_190948_lat64_lr0.0003_bs64_st176_nm128_fft2048_hop512
Dataset: 8732 samples from /Users/kate/untitled folder 10/UrbanSound8K/spectrograms
  params=5,516,033 train_batches=123 val_batches=14


Training:  50%|█████     | 1/2 [06:59<06:59, 419.12s/it]

Epoch   1/2 train L1=0.1879 val L1=0.1231 lr=1.50e-04 es=0/12


Training: 100%|██████████| 2/2 [13:25<00:00, 402.75s/it]


Best val L1: 0.1158 (epoch=2, ckpt=/Users/kate/untitled folder 10/s26_genai_project/experiments/cae/cae_nb_001_20260228_190948_lat64_lr0.0003_bs64_st176_nm128_fft2048_hop512/checkpoints/cae_best.pt)
Loaded checkpoint: /Users/kate/untitled folder 10/s26_genai_project/experiments/cae/cae_nb_001_20260228_190948_lat64_lr0.0003_bs64_st176_nm128_fft2048_hop512/checkpoints/cae_best.pt


...

Epoch   1/2 train L1=0.1958 val L1=0.1216 lr=5.05e-05 es=0/12


Training: 100%|██████████| 2/2 [12:11<00:00, 365.71s/it]


Best val L1: 0.1094 (epoch=2, ckpt=/Users/kate/untitled folder 10/s26_genai_project/experiments/cae/cae_nb_006_20260228_210226_lat256_lr0.0001_bs64_st176_nm128_fft2048_hop512/checkpoints/cae_best.pt)
Loaded checkpoint: /Users/kate/untitled folder 10/s26_genai_project/experiments/cae/cae_nb_006_20260228_210226_lat256_lr0.0001_bs64_st176_nm128_fft2048_hop512/checkpoints/cae_best.pt


Griffin-Lim: 100%|██████████| 873/873 [08:54<00:00,  1.63it/s]


  best_val=0.1094
  eval logmel_l1=0.1094 mr_stft=2.1259 si_sdr=-45.35 fad=14051.1832
  artifacts -> /Users/kate/untitled folder 10/s26_genai_project/experiments/cae/cae_nb_006_20260228_210226_lat256_lr0.0001_bs64_st176_nm128_fft2048_hop512
Completed runs: 6
Summary CSV: /Users/kate/untitled folder 10/s26_genai_project/experiments/cae/summary.csv
Summary JSON: /Users/kate/untitled folder 10/s26_genai_project/experiments/cae/summary.json


In [13]:
summary_csv = EXPERIMENTS_DIR / 'summary.csv'
if summary_csv.exists():
    summary_df = pd.read_csv(summary_csv)
    cols = [
        c
        for c in [
            'run_name', 'latent_dim', 'embed_dim', 'base_ch', 'lr', 'batch_size',
            'spec_t', 'n_mels', 'n_fft', 'hop_length',
            'best_val', 'logmel_l1', 'mr_stft', 'si_sdr', 'fad',
        ]
        if c in summary_df.columns
    ]
    display(summary_df[cols].sort_values('best_val').reset_index(drop=True))
else:
    print('No summary.csv yet. Run experiments first.')


,run_name,latent_dim,embed_dim,base_ch,lr,batch_size,spec_t,n_mels,n_fft,hop_length,best_val,logmel_l1,mr_stft,si_sdr,fad
0,cae_nb_005_20260228_204016_lat256_lr0.0003_bs6...,256,32,32,0.0003,64,176,128,2048,512,0.108130,0.108130,2.095836,-44.041294,41224.846467
1,cae_nb_006_20260228_210226_lat256_lr0.0001_bs6...,256,32,32,0.0001,64,176,128,2048,512,0.109409,0.109409,2.125901,-45.352165,14051.183184
2,cae_nb_003_20260228_195542_lat128_lr0.0003_bs6...,128,32,32,0.0003,64,176,128,2048,512,0.109917,0.109917,2.102456,-46.794003,31312.655316
3,cae_nb_001_20260228_190948_lat64_lr0.0003_bs64...,64,32,32,0.0003,64,176,128,2048,512,0.115757,0.115757,2.117707,-45.687321,18077.429474
4,cae_nb_004_20260228_201757_lat128_lr0.0001_bs6...,128,32,32,0.0001,64,176,128,2048,512,0.118986,0.118986,2.377906,-55.257423,9289.373468
5,cae_nb_002_20260228_193331_lat64_lr0.0001_bs64...,64,32,32,0.0001,64,176,128,2048,512,0.129582,0.129582,2.573899,-48.503105,6119.349129


In [14]:
%load_ext tensorboard
%tensorboard --logdir experiments/cae
